# Final project - Joel A. Mercado-Diaz
## ECEV 32000 - Introduction to Scientific Computing
## Prof. Stefano Allesina, PhD.
## Teaching assistants: Matthew Michalska-Smith & Elizabeth Sander
## Winter, 2016
--------------------------------------------------------------------------------------------------------------------------------





My project consist of two parts. The first uses Python and Regular Expressions to manipulate the name of sequence identifiers in a FASTA file so they can be used in future phylogenetic analysis. The second part make use of R and several packages (ggplot2, dplyr, ggmap) which allow us visualize some of the geographic information contained in the FASTA file in a map.

This notebook file covers the complete first part of my project and the first portion of the second one, which is completed in the other markdown provided (Final_project_Mercado-Diaz_Part2.Rmd)

The files needed to run everything are stored in the folder **Data**. As you know, these should be stored in the directory from where you are running the codes. These are the files:

*Final_alignment_Mercado_Sticta2.fasta*  
*iso_3166_2_countries*


## Part 1.

#### Problem

During my last quarter I took a class called: "Reconstructing the Tree of Life". For this class we were required to submit a final project that essentially asked us to make use of some of the tools learned in class to address a particular phylogenetic problem. I decided to keep it simple and took the opportunity to use some of this new knowledge for an ongoing project I had started previously which had to do with the taxonomy and phylogenetic relationship of species within the lichen genus _Sticta_ in Puerto Rico. My plan was to generate a Maximum Likelihood tree using a FASTA file that included a set of aligned sequences that I generated from my collections of this group in the island (26), in conjuction with another set of sequences from other 393 ingroup OTUs that was provided by a collaborator who is currently doing a world revision of this group. Unfortunately, when I started to handle the data I started to confront issues. According to the TA of that class, the issue could have had to do with the naming convention that my collaborator used for his sequences. The names were excesively long, for example, run this code and check the name of the output:


In [2]:
import csv

############################################
## Liz: added '../' to the open() function, because this notebook is in the Scripts folder
## and can't get directly to Data/
############################################
with open("../Data/Final_alignment_Mercado_Sticta2.fasta", "r") as f:
    reader = csv.DictReader(f)
    i = 0
    for row in reader:
        for keys, values in row.items():
            while i < 1:
                print("LONG NAME: " + keys)
                i += 1

LONG NAME: >Sticta_aff_laminobeauvoisii_GBXXXXXX_DNAXXXX_JMDXXX_PuertoRico_Mercado_2246_ITS


These names contained a diverse amount of information, from species name, voucher specimen id, collector, etc. This is what the elements of the name had:

*Genus_species_GeneBankAccs#_DNAExtract#_Country_Collector_SpecimenID_Locus*

I was basically getting an error when using an online tool called [Find Model](http://www.hiv.lanl.gov/content/sequence/findmodel/findmodel.html) which allows me to evaluate which nucleotide substitution model best describe my  data. Also, some file formats I was contemplating using, for example _phylyp_ format, which is required as an input file for RAxML (a free software for Maximum Likelihood phylogenetic analysis), require a short (< 10 characters) name for the sequence identifiers. I decided that perhaps was worth trying finding a way to automate the name editing of the sequences to make them shorter and therefore less clumsy.

I must confess that at the end I was actually able to run a Maximum Likelihood tree by using the [Cipres computer cluster](https://www.phylo.org/). And only until recently I learned that the names lengths were not a major issue. However, my project was already underway and I learned a lot of good stuff in the process so I decided to stick with it and show the cool things I accomplished. We never now, this stuff might actually be useful for me in the future.

Below I present the steps for this part.

### Step1. Define a function that would read the raw fasta file and return a dictionary where each key is the sequence name and the value its corresponding sequence.

In [3]:
#Define function
def read_fasta_file(fasta_file_path):
    """
    Take a fasta file and return a dictionary where
    each key is the name and each value is its sequence.
    """
    with open(fasta_file_path, "r") as fasta_file:
        fasta_dict = {}
        seq = ""
        ##########################################################
        ## Liz: you could use the .strip() method to remove the trailing newlines and starting > character
        ##########################################################
        name = fasta_file.readline() # To grab the first line in the file which has a name.
        for row in fasta_file:
            if row[0] == ">": # Sequence names start with this symbol which I use to identify them
                fasta_dict[name] = seq
                seq = ""
                name = row
            else:
                seq += row

        fasta_dict[name] = seq

    return fasta_dict

Now, lets see how this dictionary looks:

In [5]:
###################################
## Liz: also prepended ../ here
####################################
fasta_dict = read_fasta_file("../Data/Final_alignment_Mercado_Sticta2.fasta") # Call newly created function
list(fasta_dict.items())[0]

('>Sticta_gallowayana_KC732496_DNA4959_MON103_Colombia_Moncada_4637_ITS\n',
 'CGTAGGTGAACCTGCGGAAGGATCATTATCGCG--AGAGGCGTCCG----CGCCTCGGGGG------CTCCGG----CCCCCC---ACTT--CGC-A-CCC-TGTGGCT-ACT-GCACT-GGT--GTTGCTTTGGCGGCGCGTGG-CCGCC---GGAGGA---CCCGTC--AAACC-TC-TCG-AATC-G-ATGTCGTCCGAGTACC-A-TCA--TAATCGAA-TAAAAA-CTTTCAACAACGGATCTCTTGGTTCTGGCATCGATGAAGAACGCAGCGAAATGCGATAAGTAATGTGAATTGCAGAATTCAGTGAATCATCGAATCTTTGAACGCACATTGCGCCCCTTGGCATTCCGAGGGGCATACCTGTTCGAGCGTC---ATTGCA-CCCC-TCGAG----CCCTCTTT-------GGCTTGGTGTTGAGCTGC---GCGT---CCCTC------GGGACGGGCTCGAACGGCAGTGGC-GGTCCGG-CGTGCG-GGCAAGTGC-AGT-ACG-CAT-------------AAA-GCATTCGCTTG-AACGGCGCG--CCCGGG-TCCGGCCAGAC--AA----CAGCCTCCCC------GCAA------------------------TC--AACCCGTTTGACCTCGGATCAGGTAGGGATACCCGCTGAACTTAAGCATA\n')

### Step 2. Write a short function to get 3-letter codes for countries names
One of the the parts that I was interested in reducing was the country name. These are pretty long. For this I wanted to exchange the long names for 3-letter code format names. To do this, I obtained from [datahub](https://datahub.io/dataset/iso-3166-1-alpha-2-country-codes/resource/9c3b30dd-f5f3-4bbe-a3cb-d7b2c21d66ce) a file that contained the names of all countries, their corresponding 3-letter codes, and other data. Since composite country names in this file were separated by a space (e.g. New Zealand) and the country names in my file were not, the function also needed to remove this space. Also, some "countries" in my FASTA file were not actual countries, but treated as independent political entities (e.g. Galapagos). Therefore I had to manually add some extra records and create new 3-letter codes to the file to account for these names.

In [6]:
##########################################
## Liz: Manually adding entries works if there aren't many extra cases,
## but in the future you could try automatically generating new codes
## based on the full country name (grabbing first three letters, etc).
##########################################
def get_country_abbr_dict(country_csv_path):
    '''
    Read table containing three letter codes for countries (file: iso_3166_2_countries.csv)
    and create dictionary with long names as keys and 3-letter codes as values
    '''
    with open(country_csv_path, "r") as f:
        reader = csv.DictReader(f) 
        country_abbr_dict = {}
        for row in reader:
            name_w_spaces = row['Common Name']
            name_no_space = ''.join(name_w_spaces.split()) # This variable executes a function that removes the spaces whenever
                                                           # they are present
            country_abbr_dict[name_no_space] = row['ISO 3166-1 3 Letter Code']  # Creates the dictionary
    
    return country_abbr_dict

Lets take a peek a this dictionary:

In [7]:
########################################
## Liz: added ../
#######################################
country_abbr_dict = get_country_abbr_dict("../Data/iso_3166_2_countries.csv") # Call newly created function
country_abbr_dict

{'Abkhazia': 'GEO',
 'Afghanistan': 'AFG',
 'Aland': 'ALA',
 'Alaska': 'ALK',
 'Albania': 'ALB',
 'Algeria': 'DZA',
 'AmericanSamoa': 'ASM',
 'Andorra': 'AND',
 'Angola': 'AGO',
 'Anguilla': 'AIA',
 'AntiguaandBarbuda': 'ATG',
 'Argentina': 'ARG',
 'Armenia': 'ARM',
 'Aruba': 'ABW',
 'Ascension': 'ASC',
 'AshmoreandCartierIslands': 'AUS',
 'Australia': 'AUS',
 'AustralianAntarcticTerritory': 'ATA',
 'Austria': 'AUT',
 'Azerbaijan': 'AZE',
 'Azores': 'AZO',
 'Bahamas,The': 'BHS',
 'Bahrain': 'BHR',
 'BakerIsland': 'UMI',
 'Bangladesh': 'BGD',
 'Barbados': 'BRB',
 'Belarus': 'BLR',
 'Belgium': 'BEL',
 'Belize': 'BLZ',
 'Benin': 'BEN',
 'Bermuda': 'BMU',
 'Bhutan': 'BTN',
 'Bolivia': 'BOL',
 'BosniaandHerzegovina': 'BIH',
 'Botswana': 'BWA',
 'BouvetIsland': 'BVT',
 'Brazil': 'BRA',
 'BritishAntarcticTerritory': 'ATA',
 'BritishIndianOceanTerritory': 'IOT',
 'BritishSovereignBaseAreas': '',
 'BritishVirginIslands': 'VGB',
 'Brunei': 'BRN',
 'Bulgaria': 'BGR',
 'BurkinaFaso': 'BFA',
 'Buru

### Step 3. Write a function that takes as arguments the fasta dictionary with names and sequences and the dictionary with abbreviated names and return a dictionary with shortened names in keys and corresponding sequences as values

This was perhaps the hardest part and the most important. I wanted to use different naming conventions for different parts of the names, therefore I had to use RegEx to separate the parts and treat them independently. There might be simpler ways to do them, but I really wanted to practice my RegEx!

In [10]:
#Get package needed
import re

def get_short_name_fasta_dict(fasta_dict, country_abbr_dict):

    """
    Write a function that takes the fasta dictionary created in Step 1, the dictionary with abbreviated names from
    Step 2, shortens the names and retuns a dictionary with shortened names keys and sequences as values
    """

#First, use RegEx to extract different sections of the long names to facilitate data manipulation.

    short_name_fasta_dict = {} # Create empty dictionary
    for name, seq in fasta_dict.items():
        ###############################################
        ## Liz: Generally good practice to not name things "tmp" when possible
        ## Could use "first_name_part", "middle_name_part", etc.
        ###############################################
        tmp = re.findall(r'^>.*?(?=_GB|_AF|_AB|_EU|_KC|_AY|_DQ)', name) # This RegEx extract the first portion of the name, until
                                                                        # it hits one of these initials (e.g. GB)
        tmp2 = re.findall(r'[^_\W]+_[^_\W]+_[^_\W]+_[^_\W]+$', name) # This RegEx extract the latter 4 words separated by 
                                                                     # underscores at the end of the name. Country name is
                                                                     # the first element in this section
        tmp3 = re.findall(r'GBXXXXXX\_.+?(?=_)_.+?(?=_)|KC\d+\_.+?(?=_)_.+?(?=_)|' \
                      'AY\d+\_.+?(?=_)_.+?(?=_)|DQ\d+\_.+?(?=_)_.+?(?=_)|' \
                      'AF\d+\_.+?(?=_)_.+?(?=_)|AB\d+\_|EU\d+\_.+?(?=_)_.+?(?=_)', name) # This an extremely long RegEx 
                                                # (There must be a way to shorten this, I know) that I used extract the middle  
                                                # portion of the name. After doing the exercise I realized I didn't needed to use 
                                                # this for now for my renaming purposes, however since it was a lot of work
                                                # I really wanted to leave it in! :)
                                                ###########################
                                                ## Liz: I sure know that feeling!
                                                ###########################

# Then, create short names for all scientific names in the dataset. Use the variable created in the previous step which contains
# only the species names (tmp)

        split_names = tmp[0].split('_')   # Separate scientific names using underscores
        base_name = split_names[0][0:2] + "_" + split_names[1][0:3] # Select the first letter of the first word, adds a new
                                                                    # underscore and the first three letters of the second word.
                                                                    # All names had to have this
        
# Since scientific names in the file have up to 5 words separated by underscores (e.g. Sticta_aff_beauvoisii_1), I used 
# conditional statements to create the new abbreviated scientific names based on their length. After the second word, I used
# two letters to identify subsequent words in the names.
#############################
## Liz: you could handle this a little more elegantly with a for loop, which would handle any number of words
##for i, sci_name in enumerate(split_names):
##    if i == 0:
##        final_name = sci_name[0:2]
##    elif i == 1:
##        final_name = final_name + sci_name[0:3]
##    else:
##        final_name = final_name + sci_name[0:2]
##############################
        if len(split_names) == 2:
            final_name = base_name
        elif len(split_names) == 3:
            final_name = base_name + "_" + split_names[2][0:2]
        elif len(split_names) == 4:
            final_name = base_name + "_" + split_names[2][0:2] + "_" + split_names[3][0:2] 
        else:
            final_name = base_name + "_" + split_names[2][0:2] + "_" + split_names[3][0:2] + "_" + split_names[4][0:2]

# Now I need to substitute the country name in the original sequence ID with the 3 letter abbreviation of that country. To do
# this I will use the previously created dictionary "country_abbr_dict" which is in this format {country name: 'abbreviation'}
 
        sample_info = tmp2[0].split('_') # creates list with all info categories splitted. Recall
                                         # that the first element in tmp2 has the country names
        country_name = sample_info[0] # extract country names
        country_name_abbr = country_abbr_dict[country_name] # Use variable "country name" as index to the country_abbr_dict
                                                        # dictionary to get the country abbrev. for each sample
        sample_info[0] = country_name_abbr # replace long country names in the lists with the corresponding abbreviations
        sample_info_tmp = sample_info[0:3] # removes the string "ITS" from the lists 
        final_sample_info = "_".join(sample_info_tmp) # rejoins the abbreviations with the rest separated with an underscore
    
# Merge the shortened species name (final_name) with the new abbreviated sample info
    
        final_ID = final_name + "_" + final_sample_info

# Now I need to create a new dictionary that has the new short names along with their corresponding sequences

        short_name_fasta_dict[final_ID] = seq
    
    return short_name_fasta_dict

Now lets take a quick look at how this new dictionary looks:

In [11]:
short_name_fasta_dict = get_short_name_fasta_dict(fasta_dict, country_abbr_dict) # Call newly created function
list(short_name_fasta_dict.items())[0]

('>S_glo_COL_Moncada_4747',
 'CGTAGGTGAACTTGCGGAAGGATCATTACCGAG--AGAGGCGTCCATGCACGCCTCGGGGG------CCCCGG----CCCCTA---TCTT--CGC-A-CCC-CGTGCAT-ACC-GTACC-GGT--GTTGCTTTGGCGGCGCGCAG-CCGCC---GGAGA----CCCGTC--AAACC-TC-CCG-TATC-G-ATGTCGTCCGAGTCCC-A-TCGTATAATCGAA-TAAAAA-CTTTCAACAACGGATCTCTTGGTTCTGGCATCGATGAAGAACGCAGCGAAATGCGATAAGTAATGTGAATTGCAGAATTCAGTGAATCATCGAATCTTTGAACGCACATTGCGCCCCTTGGTATTCCGAGGGGCATACCTGTCCGAGCGTC---ATTGCA-CCCC-TCGAG----CCCTCTCC-------GGCTTGGTGTTGAGCCGC---GCGT--CCCCCC------GGGACGGGCTCGAACGGCAGTGGC-GGTCCGG-CGCGCC-TGCAAGTGC-AGT-AGG-CAT---------------A-CCATCCGCTTG-AGCGGCGCG--CCCGGG-TCCGGCCAGAC--AA----CAGCCTCCCC------GCAA------------------------TC--AAACCGTCTGACCTCGGATCAGGTAGGGATACCCGCTGAACTTAAGCATA\n')

### Step 4. Create a function that writes the newly created dictionary in Step 3 to a file

This last part aims to create a text file in the FASTA file format that may be used later for future phylogenetic analysis. It takes as input the dictionary created in Step 3.

In [12]:
def create_new_seq_file (short_name_fasta_dict):
    '''
     Send new dictionary to file
    '''
    #####################
    ## Liz: I might let this output file name be an additional argument to the function, maybe using
    ## file_modified_fasta as a default
    #####################
    with open("file_modified_fasta", "w") as f: # You can change here the name of the new file
        for name, seq in short_name_fasta_dict.items():
            f.write('{0}\n{1}'.format(name, seq)) # Puts the name in the first line and the corresponding sequences just below

Lets take a look at the first record in the new file

In [13]:
create_new_seq_file(short_name_fasta_dict) # Call the newly created function

with open("file_modified_fasta", "r") as f:
    reader = csv.DictReader(f)
    i = 0
    for row in reader:
        for keys, values in row.items():
            while i < 1:
                print(keys, values)
                i += 1

>S_glo_COL_Moncada_4747 CGTAGGTGAACTTGCGGAAGGATCATTACCGAG--AGAGGCGTCCATGCACGCCTCGGGGG------CCCCGG----CCCCTA---TCTT--CGC-A-CCC-CGTGCAT-ACC-GTACC-GGT--GTTGCTTTGGCGGCGCGCAG-CCGCC---GGAGA----CCCGTC--AAACC-TC-CCG-TATC-G-ATGTCGTCCGAGTCCC-A-TCGTATAATCGAA-TAAAAA-CTTTCAACAACGGATCTCTTGGTTCTGGCATCGATGAAGAACGCAGCGAAATGCGATAAGTAATGTGAATTGCAGAATTCAGTGAATCATCGAATCTTTGAACGCACATTGCGCCCCTTGGTATTCCGAGGGGCATACCTGTCCGAGCGTC---ATTGCA-CCCC-TCGAG----CCCTCTCC-------GGCTTGGTGTTGAGCCGC---GCGT--CCCCCC------GGGACGGGCTCGAACGGCAGTGGC-GGTCCGG-CGCGCC-TGCAAGTGC-AGT-AGG-CAT---------------A-CCATCCGCTTG-AGCGGCGCG--CCCGGG-TCCGGCCAGAC--AA----CAGCCTCCCC------GCAA------------------------TC--AAACCGTCTGACCTCGGATCAGGTAGGGATACCCGCTGAACTTAAGCATA


Voila! File is ready for use. Check the newly created text file in the directory.

## Part 2.
So I decided to keep playing with the data and thought that it would be neat to create a map that show points over every country that is represented in the sequences dataset. This can be later added to a future publication. To make it a little more complicated, I did some research and found a way to make the size of the points propotional to the number of times the country appears in the dataset. This would give us a visual idea if there are countries that might be overepresented.

### Step 1. Create table with identifiers and corresponding countries
The first thing to do was writing a function that created a table containing the unique identifier of each sequence in one column, extract the country and add it to another column. To do this I created a new function that actually recycled and modified some of the code I already did for the previous part!

In [14]:
def write_geo_table(fasta_dict):
    '''
    Create table with identifiers in one column and countries in the second column
    '''
    with open("geo_table.csv", "w") as geo_table:
        for name, seq in fasta_dict.items():
            stripped_name = name.strip("\n") # Strip by new line!
            tmp = re.findall(r'[^_\W]+_[^_\W]+_[^_\W]+_[^_\W]+$', name) # Recycled RegEx to get the portion with the country name
            country = tmp[0].split("_")[0] # Split the name and indexing for the first element of the name
            geo_table.write('{},{}\n'.format(stripped_name, country)) # Write the new table in to the file geo_table.tsv

Now lets see how this table looks

In [15]:
write_geo_table(fasta_dict) # Call the newly created function

import pandas # Use pandas for better visualization of the table

data = pandas.read_csv('geo_table.csv', header=None)
 
data.head(n=10)

ImportError: No module named 'pandas'

Now with this table we are ready to keep working on R!

Open the Rmd file included in the **Data** folder to continue...